# Analytics - Product Quality

#### Date: 2022/01

#### SUMMARY:

- This notebook represents the project quality analysis of the date exposed right above. 

### TEAM: Alectrion

##### Semester: 2022/01
##### Professor: Hilmer Neri

##### Members:

- Guilherme Leal
- João Pedro Soares
- Lucas Alexandre
- Matheus Estanislau
- Moacir Mascarenha
- Igor Silva de Paiva
- João Pedro Alves Machado
- Mário Vinícius
- Lucas Heler Lopes
- Pedro Siqueira
- Wildemberg Sales da Silva Junior

### LIBRARIES

In [1]:
# Deal with data
import pandas as pd
import json
from glob import glob
import os
import re

# Deal with visualization
import matplotlib.pyplot as plt

# Deal with time
import datetime

### GRAPH SETTINGS

In [2]:
fig = plt.figure(figsize=(20, 10))

<Figure size 1440x720 with 0 Axes>

### DATAFRAME SETTINGS

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

#### Replace your semester, project name, repository name, and the programming language extension

In [4]:
language = [['fga-eps-mds-2022-1-Alectrion-UserAPI', 'js'],
            ['fga-eps-mds-2022-1-Alectrion-EquipamentApi', 'js'],
            ['fga-eps-mds-2022-1-Alectrion-FrontEnd', 'js']]

repos_language = {}

for item in language:
    repos_language[f"{item[0]}"] = item[1]

### SonarCloud

##### Path to the folder with all your jsons

In [5]:
jsons = glob('analytics-raw-data/*.json') # add your path here 
print (jsons)

['analytics-raw-data/fga-eps-mds-2022-1-Alectrion-UserAPI-08-29-2022-02-14-01-v1.0.0.json', 'analytics-raw-data/fga-eps-mds-2022-1-Alectrion-FrontEnd-09-08-2022-02-35-46-v3.0.0.json', 'analytics-raw-data/fga-eps-mds-2022-1-Alectrion-FrontEnd-09-09-2022-06-37-07-pr24.json', 'analytics-raw-data/fga-eps-mds-2022-1-Alectrion-EquipamentApi-09-12-2022-21-56-09-pr4.json', 'analytics-raw-data/fga-eps-mds-2022-1-Alectrion-UserAPI-09-06-2022-00-45-24-v2.0.0.json', 'analytics-raw-data/fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-2022-01-37-27-v1.0.0.json', 'analytics-raw-data/fga-eps-mds-2022-1-Alectrion-FrontEnd-09-06-2022-00-54-12-v2.0.0.json']


In [6]:
def read_json(json_path):
    
    with open(json_path) as json_file:
        json_obj = json.load(json_file)
   
    return json_obj

def create_base_component_df(json_list):
    
    df = pd.DataFrame()
    for i in json_list:

        base_component = read_json(i)
    
        base_component_data = base_component['baseComponent']['measures']

        base_component_df = pd.DataFrame(base_component_data)

        base_component_df['filename'] = os.path.basename(i)

        df = df.append(base_component_df, ignore_index=True)
        
    # Replace the UnB semester with yours.
    aux_df = df['filename'].str.split(r"fga-eps-mds-2022-1-(.*?)-(.*?)-(.*?)-(.*?)-pr(.*?).json", expand=True)

    df['repository'] = aux_df[2]

    df['version'] = aux_df[5]

    df = df.sort_values(by=['repository', 'version'])
    
    return df

#### Create base component dataframe

In [7]:
base_component_df = create_base_component_df(jsons)

KeyError: 'baseComponent'

In [51]:
base_component_df.head(10)

,metric,value,bestValue,filename,repository,version
8,coverage,59.6,False,fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-20...,FrontEnd,1.0.0
9,complexity,91,NaN,fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-20...,FrontEnd,1.0.0
10,duplicated_lines_density,0.0,True,fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-20...,FrontEnd,1.0.0
11,functions,65,NaN,fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-20...,FrontEnd,1.0.0
12,security_rating,1.0,True,fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-20...,FrontEnd,1.0.0
13,ncloc,1354,NaN,fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-20...,FrontEnd,1.0.0
14,files,39,NaN,fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-20...,FrontEnd,1.0.0
15,comment_lines_density,1.9,False,fga-eps-mds-2022-1-Alectrion-FrontEnd-08-29-20...,FrontEnd,1.0.0
0,coverage,55.1,False,fga-eps-mds-2022-1-Alectrion-UserAPI-08-29-202...,UserAPI,1.0.0
1,complexity,106,NaN,fga-eps-mds-2022-1-Alectrion-UserAPI-08-29-202...,UserAPI,1.0.0


#### Create dataframe per file

In [52]:
metric_list = ['files',
               'functions',
               'complexity',
               'comment_lines_density',
               'duplicated_lines_density',
               'coverage',
               'ncloc',
               'tests',
               'test_errors',
               'test_failures',
               'test_execution_time',
               'security_rating']

len(metric_list)

12

In [57]:
def metric_per_file(json):
    
    file_json = []
    
    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)
            
    return file_json

def generate_file_dataframe_per_release(metric_list, json, language_extension):
    
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)
    
    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass
        
    df.reset_index(inplace = True)
    df = df.rename({'index': 'path'}, axis=1).drop(['files'], axis=1)

    return df

def create_file_df(json_list):
    
    df = pd.DataFrame()

    for i in json_list:

        file_component = read_json(i)
        
        file_component_data = metric_per_file(file_component)

        file_name = os.path.basename(i)

        file_repository = re.split(r'-(\d+-\d+-\d+-\d+-\d+-\d+)-pr(.*?).json', file_name)[0]

        file_language = repos_language[f"{file_repository}"]

        file_component_df = generate_file_dataframe_per_release(metric_list, file_component_data, language_extension = file_language)

        file_component_df['filename'] = os.path.basename(i)

        df = df.append(file_component_df, ignore_index=True)
        
    # Replace the UnB semester with yours.
    aux_df = df['filename'].str.split(r"-fga-eps-mds-2022-1-(\d+-\d+-\d+-\d+-\d+-\d+)-pr(.*?).json", expand=True)

    df['repository'] = aux_df[0]

    df['version'] = aux_df[2]

    df = df.sort_values(by=['repository', 'version'])

    return df

In [58]:
file_component_df = create_file_df(jsons)
file_component_df.repository.unique()

KeyError: 'components'

#### Removing NaN

In [56]:
file_component_df = file_component_df.dropna(subset=['functions', 'complexity','comment_lines_density', 'duplicated_lines_density', 'coverage' ])

NameError: name 'file_component_df' is not defined

        #### Create dataframe per repository

In [55]:
# Example. You must replace repo1, repo1,..., for your repository's names

repo1_df = file_component_df[file_component_df['repository'] == 'Alectrion-EquipamentApi']
repo2_df = file_component_df[file_component_df['repository'] == 'Alectrion-UserAPI']
repo3_df = file_component_df[file_component_df['repository'] == 'Alectrion-FrontEnd']
...

NameError: name 'file_component_df' is not defined

In [ ]:
def _ncloc(df):
    ncloc = 0
    for each in df['ncloc']:
        n = 0
        # try to cast the current ncloc value to int, if the value is NaN/Null, consider it as zero.
        try:
            n = int(each)
        except ValueError:
            n = 0
        ncloc += n

    return ncloc

### Measure calculations according Q-Rapids quality model

# Quality Aspect - Maintainability
## Factor - Code Quality

##### COMPLEXITY

In [ ]:
def m1(df):
    
    density_non_complex_files = len(df[(df['complexity'].astype(float)/df['functions'].astype(float)) < 10])/len(df)
    
    return density_non_complex_files

##### COMMENTS

In [ ]:
def m2(df):
    
    density_comment_files = len(df[(df['comment_lines_density'].astype(float) > 10) & (df['comment_lines_density'].astype(float) < 30)])/len(df)
    
    return density_comment_files

##### DUPLICATIONS

In [ ]:
def m3(df):
    
    duplication = len(df[(df['duplicated_lines_density'].astype(float) < 5)])/len(df)
    
    return duplication

# Quality Aspect - Reliability
## Factor - Testing Status

##### Passed tests

In [ ]:
def m4(df):

    passed_tests = (float(df['tests']) - (float(df['test_errors']) + float(df['test_failures']))) / float(df['tests'])

    return passed_tests

##### Fast test builds

In [ ]:
def m5(df):

    density_fast_test_builds = len(df[(df['test_execution_time'].astype(float)) < 300]) / sum(df['tests'].astype(float))

    return density_fast_test_builds

##### Test coverage

In [ ]:
def m6(df):

    density_test_coverage = len(df[(df['coverage'].astype(float) > 60)]) / len(df)

    return density_test_coverage

### Calculate m1, m2, m3, m4, m5 and m6 for each repository

In [ ]:
def create_metrics_df(df):
    
    version_vec = df['version'].unique()
    
    m1_list = []
    m2_list = []
    m3_list = []
    m4_list = []
    m5_list = []
    m6_list = []

    ncloc_list = []
    repository_list = []
    version_list = []
    
    for version in version_vec:

        version_df = df[df['version'] == version]

        m1_list.append(m1(version_df))
        m2_list.append(m2(version_df))
        m3_list.append(m3(version_df))
        m4_list.append(m4(version_df))
        m5_list.append(m5(version_df))
        m6_list.append(m6(version_df))

        ncloc_list.append(_ncloc(version_df))
        repository_list.append(version_df['repository'].iloc[0])
        version_list.append(version)
        
    metrics_df = pd.DataFrame({'m1': m1_list,
                               'm2': m2_list,
                               'm3': m3_list,
                               'm4': m4_list,
                               'm5': m5_list,
                               'm6': m6_list,
                               'repository': repository_list, 
                               'version': version_list,
                               'ncloc': ncloc_list})
        
    return metrics_df

In [ ]:
repo1 = create_metrics_df(repo1_df)
repo2 = create_metrics_df(repo2_df)
repo3 = create_metric_df(repo3_df)
...

### Data visualization

- You must do this for each of your repositories

In [ ]:
plt.plot(repo1['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m6'], linewidth=3, marker='o', markersize=10)

In [ ]:

plt.plot(repo2['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m6'], linewidth=3, marker='o', markersize=10)

...

### Quality factor and aspect aggregation

- You must do this for each of your repositories

In [ ]:
psc1 = 1
psc2 = 1
pc1 = 0.5
pc2 = 0.5
pm1 = 0.33
pm2 = 0.33
pm3 = 0.33
pm4 = 0.25
pm5 = 0.25
pm6 = 0.5

repo1['code_quality'] = ((repo1['m1']*pm1) + (repo1['m2']*pm2) + (repo1['m3']*pm3)) * psc1
repo2['code_quality'] = ((repo2['m1']*pm1) + (repo2['m2']*pm2) + (repo2['m3']*pm3)) * psc1
...

repo1['testing_status'] = ((repo1['m4']*pm4) + (repo1['m5']*pm5) + (repo1['m6']*pm6)) * psc2
repo2['testing_status'] = ((repo2['m4']*pm4) + (repo2['m5']*pm5) + (repo2['m6']*pm6)) * psc2
...

In [ ]:
plt.plot(repo1['code_quality'], linewidth=3, marker='o', markersize=5)
plt.plot(repo2['code_quality'], linewidth=3, marker='o', markersize=5)
...

In [ ]:

plt.plot(repo1['testing_status'], linewidth=3, marker='o', markersize=5)
plt.plot(repo2['testing_status'], linewidth=3, marker='o', markersize=5)
...


In [ ]:
metrics_df = pd.concat([repo1, repo2, ...], ignore_index=True)

metrics_df['Maintainability'] = metrics_df['code_quality'] * pc1
metrics_df['Reliability'] = metrics_df['testing_status'] * pc2
metrics_df['total'] = metrics_df['Maintainability'] + metrics_df['Reliability']

In [ ]:
metrics_df

In [ ]:
plt.plot(metrics_df['Maintainability'], linewidth=3, marker='o', markersize=5)
plt.plot(metrics_df['Reliability'], linewidth=3, marker='o', markersize=5)

plt.ylim(0.1,.7)

### You must do the total plot and the statics analysis for the repository with more versions.

In [ ]:
# Building descriptive statistics dataframe. You must replace YourRepoName for your repository name with more product versions.

more_versions_repo = metrics_df[metrics_df['repository'] == 'YourRepoName']

def get_characteristc_stats(repo_series):
    return {
        'mean': repo_series.mean(),
        'mode': repo_series.mode(),
        'median': repo_series.median(),
        'std': repo_series.std(),
        'var': repo_series.var(),
        'min': repo_series.min(),
        'max': repo_series.max()
    }

maintainability_stats = pd.DataFrame(get_characteristc_stats(more_versions_repo["Maintainability"]),
                                     columns=['mean', 'mode', 'median', 'std', 'var', 'min', 'max'])

reliability_stats = pd.DataFrame(get_characteristc_stats(more_versions_repo["Reliability"]),
                                 columns=['mean', 'mode', 'median', 'std', 'var', 'min', 'max'])

In [ ]:
print(maintainability_stats)

In [ ]:
print(reliability_stats)

### Plotting the aggregated quality characteristic indicator

In [ ]:
plt.figure(figsize=(20, 10))

# boxplot
plt.boxplot([more_versions_repo['Maintainability'], more_versions_repo['Reliability']],
labels=['Maintainability', 'Reliability'])

### Plotting the aggregated repository quality indicator

In [ ]:
plt.plot(more_versions_repo['total'], linewidth=3, marker='o', markersize=5)

plt.ylim(.1,1)

In [ ]:
# DATE FORMAT: MM-DD-YYYY-HH:MM:SS
currentDateTime = datetime.datetime.now().strftime("%m-%d-%Y-%H:%M:%S")

metrics_df.to_excel('data/fga-eps-mds-2022-1-Alectrion-{}.xlsx'.format(currentDateTime), index = False)

metrics_df.to_csv('data/fga-eps-mds-2022-1-Alectrion-{}.csv'.format(currentDateTime), index = False)